In [6]:
import random
import math
import json
import textwrap
from operator import itemgetter
import numpy as np

import sys
sys.path.insert(0, '../')

#ParticleDataTool gives us the decay channels
from ParticleDataTool import ParticleDataTool as pdt
pythia = pdt.PYTHIAParticleData(file_path='ParticleData.ppl', use_cache=True)
#Sibyll = pdt.SibyllParticleTable()
#DpmJet = pdt.DpmJetParticleTable()
#QGSJet = pdt.QGSJetParticleTable()
#UrQMD = pdt.UrQMDParticleTable()

#PYPDT gives us info about mass, charge, lifetime 
import pypdt
tbl = pypdt.ParticleDataTable()

#particle_extra_info gives us information about composition, interactions, type,...
import os
filename = '../particle_extra_info/part_extra_info.json'
particle_extra_info = json.load(open(filename))

#Part_dict is a dictornary to access pdgid from name
part_dict = {}
for p in tbl:
    part_dict[p.name]= p.id

    
#apdgid accesses part_dict processing particles that are their own antiparticle
def apdgid(partid):
    partid = str(partid)
    if pypdt.get(partid) != None:
        return pypdt.get(partid)
    else:
        if partid[0] == '-':
            return pypdt.get(partid[1:])
        else:
            return ValueError("Id not found")

        
def renormalize(n):
    range1=[-13,14]
    range2=[1e-3,5]
    delta1 = range1[1] - range1[0]
    delta2 = range2[1] - range2[0]
    return np.around((delta2 * (n - range1[0]) / delta1) + range2[0],2)

def magnitude(x):
    return int(math.log10(x))

In [7]:
class particle(object):
    def __new__(cls, *args, **kw):
        if args[0] in ['u','d','c','s','t','b']:
            return super(particle, cls).__new__(cls)
        else:    
            try:
                part_dict[args[0]]
                return super(particle, cls).__new__(cls)
            except:
                print "Unknown particle:", args[0]
    
    def __init__(self, name):
        self.name = name
        #attributes from PYPDT
        if name in ['u','d','c','s','t','b']:  #deal with quarks not defined
            self.pdgid = part_dict[name+'bar']
        else:
            self.pdgid = part_dict[name]  
        self.mass = pypdt.get(self.pdgid).mass  #tbl[self.pdgid].mass  #GeV
        self.charge = pypdt.get(self.pdgid).charge
        self.lifetime = pypdt.get(self.pdgid).lifetime  #ps e-12 s
        self.ctau = pypdt.get(self.pdgid).ctau  #mm
        self.width = pypdt.get(self.pdgid).width #GeV
        #attributes from ParticleDataTool
        try:
            self.decay_channels = sorted(pythia.decay_channels(self.pdgid),key=itemgetter(0), reverse=True)
        except:
            self.decay_channels = []
        #attributes from particle_extra_info
        self.composition = particle_extra_info[str(self.pdgid)]['composition']
        self.interactions = particle_extra_info[str(self.pdgid)]['interactions']
        self.latex = particle_extra_info[str(self.pdgid)]['latex']
        self.type = particle_extra_info[str(self.pdgid)]['type']
        self.family = particle_extra_info[str(self.pdgid)]['family']
        self.symbol = particle_extra_info[str(self.pdgid)]['symbol']
        self.antiparticle = particle_extra_info[str(self.pdgid)]['antiparticle']
        self.spin = particle_extra_info[str(self.pdgid)]['Spin']
        self.parity = particle_extra_info[str(self.pdgid)]['Parity']
        self.isospin = particle_extra_info[str(self.pdgid)]['Isospin']
        self.j = particle_extra_info[str(self.pdgid)]['J^P']
        self.strangeness = particle_extra_info[str(self.pdgid)]['Strangeness']
        self.charm = particle_extra_info[str(self.pdgid)]['Charm']
        self.bottomness = particle_extra_info[str(self.pdgid)]['Bottomness']
        
        #calculated attributes
        self.channel_names = self.channel_names()
        self.decay_seq = self.build_weights()[0]
        self.decay_weights = self.build_weights()[1]
    
    def channel_names(self):
        dc_names = []
        if self.decay_channels != []:
            for item in self.decay_channels:
                if item[0] != 0.0:            # do not use channels with prob = 0.0
                    part_names =[]
                    for part in item[1]:
                        part_names.append(apdgid(part).name)     # use apdgid to process particles that are their own antiparticle 
                    dc_names.append(tuple([item[0], part_names]))
            return sorted(dc_names,key=itemgetter(0), reverse=True)
        else:
            return dc_names
        
    def weighted_choice(self, seq, weights):
        assert len(weights) == len(seq)
        assert abs(1. - sum(weights)) < 1e-6
        x = random.random()
        for i, elmt in enumerate(seq):
            if x <= weights[i]:
                return elmt
            x -= weights[i]
    
    def build_weights(self):
        seq = []
        weights=[]
        for index, item in enumerate(self.decay_channels):
            if item[0] != 0.0:           # do not use channels with prob = 0.0
                seq.append(index)
                weights.append(item[0])
        return seq, weights
    
    def decay_tree_all(self, leaf = 1):
        wrapper1 = textwrap.TextWrapper(initial_indent='-'*leaf, width=70)
        wrapper2 = textwrap.TextWrapper(initial_indent='-'*(leaf+1), width=70)
        if self.channel_names !=[]:
            for index, channel in enumerate(self.channel_names):
                print wrapper1.fill('Channel {} : {}'.format(index, channel[0]))
                for part in channel[1]:
                    print wrapper2.fill(part)
                    particle(part).decay_tree_all(leaf+1)
        else: 
            print wrapper2.fill('Stable')
            
    def decay_tree_random(self, leaf=1):
        wrapper = textwrap.TextWrapper(initial_indent='-'*leaf, width=70)
        if self.channel_names !=[]:
            choice = self.weighted_choice(self.decay_seq,self.decay_weights)
            channel = self.channel_names[choice][1]
            for part in channel:
                print wrapper.fill(part)
                particle(part).decay_tree_random(leaf+1)           
        else: 
            print wrapper.fill('Stable')
        
    def next_time(self):
        return random.expovariate(1/self.lifetime)
   
    def next_time_ren(self):
        return renormalize(magnitude(self.next_time()))

        
    

In [10]:
dbar = particle('dbar')
dbar.channel_names

[]

In [13]:
dbar.lifetime

In [26]:
f = particle('gamma0')
f.mass

0.0

In [7]:
H = particle('H0')
H.decay_tree_random()

-bbar
--Stable
-bbar
--Stable


In [4]:
Z = particle('Z0')
Z.decay_tree_random()

Unknown particle: Z0


AttributeError: 'NoneType' object has no attribute 'decay_tree_random'

In [8]:
Z.decay_channels

[(0.153995, [1, -1]),
 (0.153984, [3, -3]),
 (0.152272, [5, -5]),
 (0.11942, [2, -2]),
 (0.119259, [4, -4]),
 (0.066806, [12, -12]),
 (0.066806, [14, -14]),
 (0.066806, [16, -16]),
 (0.033576, [11, -11]),
 (0.033576, [13, -13]),
 (0.0335, [15, -15])]

In [102]:
W = particle('W+')
W.decay_tree_random()

-e+
--Stable
-nu_e
--Stable


In [38]:
t = particle('tau+')
t.decay_tree_random()

-nubar_tau
--Stable
-pi0
--gamma0
---Stable
--gamma0
---Stable
-pi+
--mu+
---nu_e
----Stable
---e+
----Stable
---nubar_mu
----Stable
--nu_mu
---Stable


In [174]:
n= particle('n0')
n.decay_tree()


-Channel 0 : 1.0
--nubar_e
---Stable
--e-
---Stable
--p+
---Stable


In [99]:
e=particle('e-')
e.decay_tree_random()

-Stable


In [176]:
mu= particle('mu+')
mu.decay_tree()

-Channel 0 : 1.0
--nu_e
---Stable
--e+
---Stable
--nubar_mu
---Stable


In [100]:
pi= particle('pi+')
pi.decay_tree_random()

-mu+
--nu_e
---Stable
--e+
---Stable
--nubar_mu
---Stable
-nu_mu
--Stable


In [13]:
Z = particle('Z0')
Z.channel_names

[(0.153995, ['dbar', 'dbar']),
 (0.153984, ['sbar', 'sbar']),
 (0.152272, ['bbar', 'bbar']),
 (0.11942, ['ubar', 'ubar']),
 (0.119259, ['cbar', 'cbar']),
 (0.066806, ['nu_e', 'nubar_e']),
 (0.066806, ['nu_mu', 'nubar_mu']),
 (0.066806, ['nu_tau', 'nubar_tau']),
 (0.033576, ['e-', 'e+']),
 (0.033576, ['mu-', 'mu+']),
 (0.0335, ['tau-', 'tau+'])]

In [17]:
W = particle('W+')
W.channel_names

[(0.321369, ['dbar', 'ubar']),
 (0.320615, ['sbar', 'cbar']),
 (0.108166, ['e+', 'nu_e']),
 (0.108166, ['mu+', 'nu_mu']),
 (0.108087, ['tau+', 'nu_tau']),
 (0.016502, ['sbar', 'ubar']),
 (0.016494, ['dbar', 'cbar']),
 (0.000591, ['bbar', 'cbar']),
 (1e-05, ['bbar', 'ubar'])]

In [50]:
dbar = particle('dbar')
dbar.channel_names()

'stable particle'

In [20]:
t = particle('tau+')
t.channel_names

[(0.25489, ['nubar_tau', 'pi0', 'pi+']),
 (0.178345, ['nubar_tau', 'e+', 'nu_e']),
 (0.173545, ['nubar_tau', 'mu+', 'nu_mu']),
 (0.108924, ['nubar_tau', 'pi+']),
 (0.09237, ['nubar_tau', 'pi0', 'pi0', 'pi+']),
 (0.089813, ['nubar_tau', 'pi+', 'pi+', 'pi-']),
 (0.044435, ['nubar_tau', 'pi0', 'pi+', 'pi+', 'pi-']),
 (0.010313, ['nubar_tau', 'pi0', 'pi0', 'pi0', 'pi+']),
 (0.008957, ['nubar_tau', 'pi+', 'K0']),
 (0.006885, ['nubar_tau', 'K+']),
 (0.004935, ['nubar_tau', 'pi0', 'pi0', 'pi+', 'pi+', 'pi-']),
 (0.004491, ['nubar_tau', 'pi0', 'K+']),
 (0.003757, ['nubar_tau', 'pi0', 'pi+', 'K0']),
 (0.003292, ['nubar_tau', 'pi+', 'pi-', 'K+']),
 (0.001762, ['nubar_tau', 'gamma0', 'pi0', 'pi+']),
 (0.001744, ['nubar_tau', 'pi0', 'pi+', 'eta0']),
 (0.001519, ['nubar_tau', 'pi+', 'K+', 'K-']),
 (0.001518, ['nubar_tau', 'pi0', 'K0', 'K+']),
 (0.001513, ['nubar_tau', 'K0', 'K+']),
 (0.001087, ['nubar_tau', 'K(L)0', 'pi+', 'K(S)0']),
 (0.000957, ['nubar_tau', 'pi0', 'pi0', 'pi0', 'pi0', 'pi+']),
 (

In [259]:
import math
import random

def nextTime(rateParameter):
    return -math.log(1.0 - random.random()) * rateParameter

In [13]:
random.expovariate(1/W.lifetime)

1.2783926824204178e-13

In [16]:
import numpy as np
lifetime_array = []
for p in tbl: 
    if p.lifetime != None:
        lifetime_array.append(p.lifetime)
lifetime_array=np.asarray(lifetime_array)
def magnitude(x):
    return int(math.floor(math.log10(x)))

for item in lifetime_array:
    print magnitude(item)

-13
6
-1
-11
-13
-13
-1
-12
-12
-1
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-11
-12
-11
-10
-5
-12
-12
-12
-12
-12
-10
-11
4
-1
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-12
-11
-10
-11
4
-12
-10
-12
-12
-12
-7
-11
-12
-12
-10
-12
-2
-12
-12
-12
1
-11
-12
-12
-12
-12
4
-11
-12
-12
-12
-9
-10
-12
-12
-11
14
-12
-12
-12
0
-9
-11
-1
-11
-1
-11
-11
-9
-10
-12
-12
-12
-12
0
-11
0
-11
0
0
-10
-1
-12
-9
-8
-12
-11
-11
-12
-12
-12
-12
-12
-12
-12
-12
-12
0
-10
-12
-12
-12
-12
-12
-12
-12
-12
-12
-11
2
-12
0
-12
-11
-12
-11
-11
-11
-12
-11
1
-12
-11
-11
-11
-12
-11
-12
-12
-11
-12
-12
-12
-12
-11
2
-12
-12
-12
-12
0
-10
-11
-11
-11
-11
-10
-11
-12
-11
-12
-12
-12
2
-11
-12
-11
-12
2
-11
-12
-12
-12
-12
-12
-12
-11
-11
-11
0
-12
-12
-8
-11
-12
-12
1
-11
-12
-12
-12
-12
-8
-12
-12
-12
-12
-10
0
-12
2
-11
2
-12
-11
-12
-12
-10
-12
-12
-12
-10
-12
-12
-11
-12
-12
-11
-11
-11
-11
-12
-12
-12
-1
-11
0
-12
-12
-11
-12
-12
-11
-1
-12
-11
-12
-9
0
-12
-12
-12
-12
-11

In [10]:
pi=particle('pi+')
pi.composition

[[u'u', u'dbar']]

In [17]:
pypdt.get(pi.pdgid).isBaryon()

False

In [32]:
pi.next_time()

7236.671284925488

In [9]:
from pypdt import pid

In [14]:
pid.hasDown(-100211)

True

In [5]:
pypdt.get(-100211).hasDown()

True

In [6]:
pypdt.get(-100211).hasUp()

True

In [7]:
pypdt.get(-100211).hasTop()

False

In [8]:
pypdt.get(-100211).hasFundamentalAnti()

False